In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
import glob
import os
import datetime
pd.set_option("max_columns", 200)



for y in ['20200703']:
    print('----------------------------------------------------------------')
    print(y)
    readPath = '/mnt/dailyRawData/logs_' + y + '_zt_52_03_day_pcap/mdL2Pcap_SH_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    logSH1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    logSH1["StockID"] = logSH1["ID"] - 1000000
    logSH1 = logSH1[["sequenceNo", "StockID", "time", "cum_volume", "cum_amount", "close",
                     "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q", "bid3q", 
                     "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", 
                     "ask2q", "ask3q", "ask4q", "ask5q", "open", "cum_tradesCnt"]]
    logSH1 = logSH1.rename(columns={"open":"openPrice", "cum_tradesCnt":"numTrades"})
        
    path1 = np.array(glob.glob('/mnt/Kevin_zhenyu/rawData/logs_' + y + '_zs_92_01_day_data/mdLog_SH_*'))[0]
    path2 = np.array(glob.glob('/mnt/Kevin_zhenyu/rawData/logs_' + y + '_zs_92_01_day_data/mdLog_SZ_*'))[0]
    path3 = np.array(glob.glob('/mnt/Kevin_zhenyu/rawData/logs_' + y + '_zt_96_04_day_96data/mdLog_SH_*'))[0]
    
    
    startTm = datetime.datetime.now()
    logSH2 = pd.read_csv(path3,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice", "numTrades"]]
    print(datetime.datetime.now() - startTm)
    logSH2["time"] = logSH2["time"].apply(lambda x: int((x.replace(':', "")).replace(".", ""))) 
    
    print('----------------------------------------------------------------')
    print('SH lv2 data:')
    in_dex = [16, 300, 852, 905]
    data1 = logSH2[~logSH2["StockID"].isin(in_dex) & (logSH2["time"] >= 91500000) & (logSH2["time"] <= 150000000) \
              & (logSH2["source"] == 13)]
    data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1["time"] >= 91500000) & (logSH1["time"] <= 150000000)]
    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "time", "numTrades"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())
    print(n1)
    print(n2)
    print(len(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique())))
    print(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique()))
    
    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & set(data2_1["StockID"].unique()))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]
    
    for cols in ["cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p","ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "openPrice"]:
        data1_1[cols] = (data1_1[cols]*10000).round(0)
    
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
        display(len(test[np.isnan(test["sequenceNo_y"])])/n1)
        display(len(test[np.isnan(test["sequenceNo_y"])]["time"].unique()))
        display(test[np.isnan(test["sequenceNo_y"])]["time"].unique())
        display(len(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique()))
        display(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(test[np.isnan(test["sequenceNo_x"])])
        display(n2-n1)
        print((n2-n1)/n1)
#     del logSH2
#     del data1
#     del data2
#     del test
#     del data1_1
#     del data2_1
    
    
    print('----------------------------------------------------------------')
    print('SH index data:')
    
    readPath = '/mnt/dailyRawData/logs_' + y + '_zt_52_03_day_pcap/mdIndexPcap_SH_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    index = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    index["StockID"] = index["ID"] - 1000000
    index = index.rename(columns={"open":"openPrice"})

    startTm = datetime.datetime.now()
    logSH = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
    print(datetime.datetime.now() - startTm)
    logSH["time"] = logSH["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
    
    in_dex = [16, 300, 852, 905]
    index = index[index["StockID"].isin(in_dex)]
    print(index["StockID"].unique())
    
    data1 = logSH[(logSH["StockID"].isin(in_dex)) & (logSH["time"] >= 91500000) & (logSH["time"] <= 150000000)]
    data2 = index[(index["time"] >= 91500000) & (index["time"] <= 150000000)]

    columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
    
    for cols in ["cum_amount", "close", "openPrice"]:
        data1_1[cols] = (data1_1[cols]*10000).round(0)
        
        
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
    if (n2 == len1) & (n1 < len1):
        display("baseline is not complete::")
        display(test[np.isnan(test["sequenceNo_x"])])
    
    columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice", "time"]
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
    if (n2 == len1) & (n1 < len1):
        display("baseline is not complete::")
        display(test[np.isnan(test["sequenceNo_x"])])
    
#     del index
#     del logSH
#     del data1
#     del data2
#     del test
#     del data1_1
#     del data2_1

    
    print('----------------------------------------------------------------')
    print('SZ lv2 data:')
    
    readPath = '/mnt/dailyRawData/logs_' + y + '_zs_***_day_pcap/mdL2Pcap_SZ_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    logSZ1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    logSZ1 = logSZ1.loc[:, ["clockAtArrival", "sequenceNo", "ID", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "open", "cum_tradesCnt"]]
    logSZ1 = logSZ1.rename(columns={"open":"openPrice", "cum_tradesCnt":"numTrades"})
    logSZ1["StockID"] = logSZ1["ID"] - 2000000

    startTm = datetime.datetime.now()
    logSZ = pd.read_csv(path2,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice", "numTrades"]]
    logSZ["time"] = logSZ["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
    print(datetime.datetime.now() - startTm)
    
    startTm = datetime.datetime.now()
    data1 = logSZ[(logSZ["time"] >= 91500000) & (logSZ["time"] < 150000000) & (logSZ["source"] == 4)]
    data2 = logSZ1[(logSZ1["time"] >= 91500000) & (logSZ1["time"] < 150000000)]

    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "numTrades", "time"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())
    print(n1)
    print(n2)
    print(len(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique())))
    print(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique()))
   
    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & set(data2_1["StockID"].unique()))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]
    for i in ["cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "openPrice"]:
        data1_1[i] = (data1_1[i] * 10000).round(0)

    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    print(datetime.datetime.now() - startTm)
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
        print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
        print(np.sort(test[np.isnan(test["sequenceNo_y"])]["time"].unique()))
        print(len(np.sort(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique())))
        print(np.sort(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique()))
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(test[np.isnan(test["sequenceNo_x"])])
        display(n2-n1)
#     del logSZ
#     del logSZ1
#     del data1
#     del data2
#     del test
#     del data1_1
#     del data2_1
    
    
    
    
    
    readPath = '/mnt/dailyRawData/logs_' + y + '_zs_***_day_pcap/mdOrderPcap_SZ_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    OrderLogSZ1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    OrderLogSZ1["SecurityID"] = OrderLogSZ1["ID"] - 2000000
    OrderLogSZ1 = OrderLogSZ1.rename(columns={"time":'TransactTime'})
    

    path1 = np.array(glob.glob('/mnt/Kevin_zhenyu/rawData/logs_' + y + '_zs_92_01_day_data/mdOrderLog_*'))[0]
    OrderLogSZ = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty"]]
    OrderLogSZ["OrderType"] = OrderLogSZ["OrderType"].apply(lambda x: str(x))
    
    OrderLogSZ1["OrderType"] = np.where(OrderLogSZ1["OrderType"] == 2, '2', np.where(
        OrderLogSZ1["OrderType"] == 1, '1', OrderLogSZ1['OrderType']))
    

    display(len(OrderLogSZ["SecurityID"].unique()))
    display(len(OrderLogSZ1["SecurityID"].unique()))
    display(len(set(OrderLogSZ["SecurityID"].unique()) - set(OrderLogSZ1["SecurityID"].unique())))
    display(set(OrderLogSZ["SecurityID"].unique()) - set(OrderLogSZ1["SecurityID"].unique()))


    sl = list(set(OrderLogSZ["SecurityID"].unique()) & set(OrderLogSZ1['SecurityID'].unique()))
    OrderLogSZ = OrderLogSZ[OrderLogSZ["SecurityID"].isin(sl)]
    OrderLogSZ1 = OrderLogSZ1[OrderLogSZ1["SecurityID"].isin(sl)]
    print(len(OrderLogSZ["SecurityID"].unique()))
    print(len(OrderLogSZ1["SecurityID"].unique()))
    
    print('----------------------------------------------------------------')
    print('SZ order data:')
    
    columns = ["ApplSeqNum", "TransactTime", "Side", 'OrderType', 'Price', 'OrderQty', "SecurityID"]
    ree = pd.merge(OrderLogSZ, OrderLogSZ1, on=columns, how="outer", validate='one_to_one')
    n1 = ree["sequenceNo_x"].count()
    n2 = ree["sequenceNo_y"].count()
    len1 = len(ree)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(ree[np.isnan(ree["sequenceNo_y"])])
        print(len(ree[np.isnan(ree["sequenceNo_y"])]))
        print(np.sort(ree[np.isnan(ree["sequenceNo_y"])]["TransactTime"].unique()))
        print(len(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique()))
        print(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("test is complete, baseline is not complete:")
        display(ree[np.isnan(ree["sequenceNo_x"])])
        print(np.sort(ree[np.isnan(ree["sequenceNo_x"])]["TransactTime"].unique()))
        print(len(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique()))
        print(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique())
        display(n2-n1)
#     del OrderLogSZ
#     del OrderLogSZ1
#     del ree
    
    
    
    
    
    readPath = '/mnt/dailyRawData/logs_' + y + '_zt_52_03_day_pcap/mdTradePcap_SH_***'
    dataPathLs = np.array(glob.glob(readPath))
    
    startTm = datetime.datetime.now()
    SH1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    SH1["SecurityID"] = SH1["ID"] - 1000000
    SH1 = SH1.rename(columns={"time":'TransactTime'})

    path1 = np.array(glob.glob('/mnt/Kevin_zhenyu/rawData/logs_' + y + '_zs_92_01_day_data/mdTradeLog_*'))[0]
    SH = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
    SH = SH[SH["exchId"] == 1]
    
    display(len(SH["SecurityID"].unique()))
    display(len(SH1["SecurityID"].unique()))
    display(len(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique())))
    display(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique()))

    
    sl = list(set(SH["SecurityID"].unique()) & set(SH1['SecurityID'].unique()))
    SH = SH[SH["SecurityID"].isin(sl)]
    SH1 = SH1[SH1["SecurityID"].isin(sl)]
    print(len(SH["SecurityID"].unique()))
    print(len(SH1["SecurityID"].unique()))

    print(SH1.columns)
    
    print('----------------------------------------------------------------')
    print('SH trade data:')
    
    SH["ExecType"] = SH["ExecType"].apply(lambda x: str(x))
    SH1["ExecType"] = 'F'
    columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum"]
    re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = re["sequenceNo_x"].count()
    n2 = re["sequenceNo_y"].count()
    len1 = len(re)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(re[np.isnan(re["sequenceNo_y"])])
        print(len(re[np.isnan(re["sequenceNo_y"])]))
        print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(re[np.isnan(re["sequenceNo_x"])])
        print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
        display(n2-n1)
#     del SH
#     del SH1
#     del re
    
    
    
    
    
    
    readPath = '/mnt/dailyRawData/logs_' + y + '_zs_***_day_pcap/mdTradePcap_SZ_***'
    dataPathLs = np.array(glob.glob(readPath))
    
    startTm = datetime.datetime.now()
    TradeLogSZ1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    TradeLogSZ1["SecurityID"] = TradeLogSZ1["ID"] - 2000000
    TradeLogSZ1 = TradeLogSZ1.rename(columns={"time":'TransactTime'})
    TradeLogSZ1["TradeBSFlag"] = 'N'
    
    startTm = datetime.datetime.now()
    path1 = np.array(glob.glob('/mnt/Kevin_zhenyu/rawData/logs_' + y + '_zs_92_01_day_data/mdTradeLog_*'))[0]

    TradeLogSZ = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
    TradeLogSZ = TradeLogSZ[TradeLogSZ["exchId"] != 1]
    print(datetime.datetime.now() - startTm)
    
    
    display(len(TradeLogSZ["SecurityID"].unique()))
    display(len(TradeLogSZ1["SecurityID"].unique()))
    display(len(set(TradeLogSZ["SecurityID"].unique()) - set(TradeLogSZ1["SecurityID"].unique())))
    display(set(TradeLogSZ["SecurityID"].unique()) - set(TradeLogSZ1["SecurityID"].unique()))


    sl = list(set(TradeLogSZ["SecurityID"].unique()) & set(TradeLogSZ1['SecurityID'].unique()))
    TradeLogSZ = TradeLogSZ[TradeLogSZ["SecurityID"].isin(sl)]
    TradeLogSZ1 = TradeLogSZ1[TradeLogSZ1["SecurityID"].isin(sl)]
    print(len(TradeLogSZ["SecurityID"].unique()))
    print(len(TradeLogSZ1["SecurityID"].unique()))

    print(TradeLogSZ1.columns)
    
    
    print('----------------------------------------------------------------')
    print('SZ trade data:')
    
    TradeLogSZ["ExecType"] = TradeLogSZ["ExecType"].apply(lambda x: str(x))
    TradeLogSZ1["ExecType"] = TradeLogSZ1["ExecType"].apply(lambda x: str(x))

    columns = ["TransactTime","ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag","TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum","OfferApplSeqNum"]
    re = pd.merge(TradeLogSZ, TradeLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = re["sequenceNo_x"].count()
    n2 = re["sequenceNo_y"].count()
    len1 = len(re)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(re[np.isnan(re["sequenceNo_y"])])
        print(len(re[np.isnan(re["sequenceNo_y"])]))
        print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(re[np.isnan(re["sequenceNo_x"])])
        print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
        display(n2-n1)
#     del TradeLogSZ
#     del TradeLogSZ1
#     del re

----------------------------------------------------------------
20200703
0:07:10.705251
0:01:48.347968
----------------------------------------------------------------
SH lv2 data:
1616
11959
0
set()
6756282
6756282
6756282
-----------------------------------------------
----------------------------------------------------------------
SH index data:
0:00:02.207875
0:00:47.063470
[ 16 300 852 905]
11399
11399
11399
11399
11399
22798


'test is not complete:'

,index_x,clockAtArrival_x,sequenceNo_x,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,index_y,clockAtArrival_y,sequenceNo_y,ID,prevClose
0,174532.0,1.593739e+15,734884.0,5.0,300,SH,91502000,0,0.000000e+00,43353249.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,175465.0,1.593739e+15,881044.0,5.0,16,SH,91504000,0,0.000000e+00,30842381.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,175471.0,1.593739e+15,883477.0,5.0,852,SH,91504000,0,0.000000e+00,64124411.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,175472.0,1.593739e+15,883478.0,5.0,905,SH,91504000,0,0.000000e+00,59742027.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,406216.0,1.593740e+15,2323877.0,5.0,16,SH,92514000,825459,1.709155e+13,31013858.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31013858.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11394,13684415.0,1.593759e+15,151670150.0,5.0,905,SH,145704000,224476957,2.147910e+15,60507217.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59899958.0,NaN,NaN,NaN,NaN,NaN
11395,13689157.0,1.593759e+15,151691511.0,5.0,16,SH,145709000,77473396,1.363263e+15,31585996.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31013858.0,NaN,NaN,NaN,NaN,NaN
11396,13689489.0,1.593759e+15,151692181.0,5.0,300,SH,145709000,287619711,4.380277e+15,44184839.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43531937.0,NaN,NaN,NaN,NaN,NaN
11397,13689490.0,1.593759e+15,151692183.0,5.0,852,SH,145709000,218052453,2.467262e+15,64780736.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64240941.0,NaN,NaN,NaN,NaN,NaN


----------------------------------------------------------------
SZ lv2 data:
0:05:48.353336
0:02:36.606939
2222
8659
0
set()
0:03:58.143666
8626101
8625807
8626101
-----------------------------------------------


'test is not complete:'

...

/home/work516/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2211

3129

0

set()

In [9]:
logSZ[(logSZ["StockID"] == 651) & (logSZ["time"] >= 102903000) & (logSZ["source"] == 4)][["StockID", "time", 
                                                                                         "cum_volume", "close",
                                                                                         "bid1p", "bid2p", "bid1q",
                                                                                         "bid2q", "ask1p", "ask2p",
                                                                                         "ask1q", "ask2q"]].head(5)

,StockID,time,cum_volume,close,bid1p,bid2p,bid1q,bid2q,ask1p,ask2p,ask1q,ask2q
4376386,651,102903000,12400705,56.87,56.86,56.85,23100,42600,56.88,56.89,5300,16700
4379467,651,102906000,12401805,56.88,56.86,56.85,26000,42600,56.88,56.89,5000,16700
4382563,651,102909000,12422705,56.86,56.86,56.85,12800,42600,56.88,56.89,200,15500
4385485,651,102912000,12425105,56.88,56.86,56.85,13100,42700,56.88,56.89,10700,15500
4388526,651,102915000,12426505,56.88,56.87,56.86,1200,18300,56.88,56.89,9400,15500


In [10]:
logSZ1[(logSZ1["StockID"] == 651) & (logSZ1["time"] >= 102903000)][["StockID", "time", 
                                                                                         "cum_volume", "close",
                                                                                         "bid1p", "bid2p", "bid1q",
                                                                                         "bid2q", "ask1p", "ask2p",
                                                                                         "ask1q", "ask2q"]].head(5)

,StockID,time,cum_volume,close,bid1p,bid2p,bid1q,bid2q,ask1p,ask2p,ask1q,ask2q
3021612,651,102903000,12400705,568700,568600,568500,23100,42600,568800,568900,5300,16700
3023413,651,102906000,12401805,568800,568600,568500,26000,42600,568800,568900,5000,16700
3026925,651,102912000,12425105,568800,568600,568500,13100,42700,568800,568900,10700,15500
3028676,651,102915000,12426505,568800,568700,568600,1200,18300,568800,568900,9400,15500
3030438,651,102918000,12428305,568700,568700,568600,600,19300,568800,568900,9200,16000


In [3]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
import glob
import os
import datetime
pd.set_option("max_columns", 200)



for y in ['20200629']:
    print('----------------------------------------------------------------')
    print(y)
    readPath = '/mnt/Kevin_zhenyu/rawData/logs_' + y + '_zs_92_01_day_data/mdLog_SH_*'
    display(np.array(glob.glob(readPath)))

----------------------------------------------------------------
20200629


array(['/mnt/Kevin_zhenyu/rawData/logs_20200629_zs_92_01_day_data/mdLog_SH_20200629_0853.csv'],
      dtype='<U84')

In [10]:
logSH[(logSH["StockID"] == 300) & (logSH["cum_volume"] >= 34684490)].head(20)

,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice
1755207,1593395648127694,27026134,5,300,SH,95404000,34684490,6.018889e+10,4113.8245,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4127.9348
1760014,1593395653128718,27083188,5,300,SH,95409000,34828420,6.034620e+10,4113.9180,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4127.9348
1764389,1593395658119052,27140709,5,300,SH,95414000,34905039,6.047542e+10,4114.2036,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4127.9348
1768822,1593395663118557,27194282,5,300,SH,95419000,34973030,6.057729e+10,4114.0006,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4127.9348
1773657,1593395668119461,27248063,5,300,SH,95424000,35063246,6.070963e+10,4114.2833,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4127.9348
1777922,1593395673118744,27303271,5,300,SH,95429000,35145514,6.079650e+10,4114.1797,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4127.9348
1782168,1593395677739683,27350168,5,300,SH,95433000,35145514,6.079650e+10,4114.1797,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4127.9348
1782382,1593395678119950,27355878,5,300,SH,95434000,35239767,6.092944e+10,4114.1774,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4127.9348
1787115,1593395683120166,27407588,5,300,SH,95439000,35319100,6.105645e+10,4114.4179,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4127.9348
1791420,1593395688119777,27462651,5,300,SH,95444000,35395925,6.117489e+10,4114.1984,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4127.9348


In [11]:
index[(index["StockID"] == 300) & (index["cum_volume"] >= 34684490)].head(20)

,clockAtArrival,sequenceNo,ID,time,cum_volume,cum_amount,close,openPrice,prevClose,StockID
96864,1593395649714093,8313892,1000300,95403370,34684490,601888875735000,41138245,41279348,41389894,300
97254,1593395659703931,8348406,1000300,95413600,34905039,604754190114000,41142036,41279348,41389894,300
97297,1593395660624955,8351317,1000300,95413600,34905039,604754190114000,41142036,41279348,41389894,300
97454,1593395664704729,8365289,1000300,95418370,34973030,605772941079000,41140006,41279348,41389894,300
97689,1593395669704164,8381726,1000300,95423870,35063246,607096312560000,41142833,41279348,41389894,300
97900,1593395674705132,8397843,1000300,95428600,35145514,607965043650000,41141797,41279348,41389894,300
98104,1593395679705454,8414237,1000300,95433360,35239767,609294421165000,41141774,41279348,41389894,300
98483,1593395689705843,8446611,1000300,95443610,35395925,611748873981000,41141984,41279348,41389894,300
98681,1593395694735518,8462875,1000300,95448370,35451055,612594822066000,41143171,41279348,41389894,300
98908,1593395699726358,8479412,1000300,95453880,35528800,613998537297000,41140701,41279348,41389894,300


In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
import glob
import os
import datetime
pd.set_option("max_columns", 200)

for y in ['20200618']:
    print('----------------------------------------------------------------')
    print(y)
    path2 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdLog_SZ_*'))[0]


    
    print('----------------------------------------------------------------')
    print('SZ lv2 data:')
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zs_96_03_day_pcap\\mdL2Pcap_SZ_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    logSZ1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    logSZ1 = logSZ1.loc[:, ["clockAtArrival", "sequenceNo", "ID", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "open", "cum_tradesCnt"]]
    logSZ1 = logSZ1.rename(columns={"open":"openPrice", "cum_tradesCnt":"numTrades"})
    logSZ1["StockID"] = logSZ1["ID"] - 2000000


    logSZ = pd.read_csv(path2,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice", "numTrades"]]
    logSZ["time"] = logSZ["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))


----------------------------------------------------------------
20200618
----------------------------------------------------------------
SZ lv2 data:
0:03:08.111037


In [9]:
logSZ1[(logSZ1["StockID"] == 2616) & (logSZ1["time"] > 103200000)][["StockID", "time", "cum_volume", "close", "bid1p", "bid2p", "ask1p", "ask2p", "bid1q", "bid2q", "ask1q", "ask2q"]].head(5)

,StockID,time,cum_volume,close,bid1p,bid2p,ask1p,ask2p,bid1q,bid2q,ask1q,ask2q
3195504,2616,103203000,2702600,116800,116800,116600,117000,117200,2100,31900,4200,700
3199510,2616,103209000,2706300,116600,116600,116500,116700,116800,30600,6700,2100,2300
3201480,2616,103212000,2707000,116700,116600,116500,116700,116800,30600,6700,3500,2300
3203381,2616,103215000,2707100,116700,116600,116500,116700,116800,30600,6700,4100,2300
3205322,2616,103218000,2707100,116700,116600,116500,116700,116800,30600,6700,4200,2300


In [8]:
logSZ[(logSZ["StockID"] == 2616) & (logSZ["time"] > 103200000) & (logSZ["source"] == 4)][["StockID", "time", "cum_volume", "close", "bid1p", "bid2p", "ask1p", "ask2p", "bid1q", "bid2q", "ask1q", "ask2q"]].head(5)

,StockID,time,cum_volume,close,bid1p,bid2p,ask1p,ask2p,bid1q,bid2q,ask1q,ask2q
4705606,2616,103203000,2702600,11.68,11.68,11.66,11.70,11.72,2100,31900,4200,700
4709176,2616,103206000,2704700,11.68,11.67,11.66,11.68,11.69,300,31900,2300,700
4712611,2616,103209000,2706300,11.66,11.66,11.65,11.67,11.68,30600,6700,2100,2300
4716071,2616,103212000,2707000,11.67,11.66,11.65,11.67,11.68,30600,6700,3500,2300
4719401,2616,103215000,2707100,11.67,11.66,11.65,11.67,11.68,30600,6700,4100,2300


In [5]:
readPath = r'\\mentos\dailyRawData\logs_20200618_zt_52_03_day_pcap\***'
dataPathLs = np.array(glob.glob(readPath))
dataPathLs

array([], dtype=float64)